In [ ]:
import numpy as np
import os.path as osp
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

ori_img = Image.open("/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/panda.jpg")
print("img shape: ", ori_img.size)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(ori_img)

random_era = transforms.RandomErasing(  p=1.0,\
                                        scale=(1.0 - 1e-5, 1.0),\
                                        ratio=(1.0, 1.0+0.00000001),\
                                        value=255)
transform = transforms.Compose([
    transforms.PILToTensor(),
    random_era,
    transforms.ToPILImage(),
])
erased_img = transform(ori_img)
plt.subplot(1, 2, 2)
plt.imshow(erased_img)

plt.show()

In [7]:
#################### VISUALIZE INFERENCE TIME ####################
import numpy as np
import random
from torchsummary import summary
from glob import glob
import torch
import time
import torch.nn as nn
import os
from os.path  import join
from sklearn import metrics
from tqdm import tqdm
from sklearn.svm import SVC
from spectrum.preprocess_data import extract_features as extract_features_spectrum
from visual_artifact.process_data import extract_features as extract_features_visual
import pickle 
from util import get_datasetname, make_outputfeature_dir, split_real_fake, get_test_path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from headpose_forensic.eval_headpose import process_training_data

saved_feature = '/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/test.pkl'

def calculate_metric(y_label, y_pred_label):
    mic_accuracy = accuracy_score(y_label, y_pred_label)
    macro_precision = precision_score(y_label, y_pred_label, average='macro')
    macro_recall = recall_score(y_label, y_pred_label, average='macro')
    macro_f1 = f1_score(y_label, y_pred_label, average='macro')
    return mic_accuracy, macro_precision, macro_recall, macro_f1

def eval_spectrum(data, model_file):
    pkl_file = open(data, 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    X = data["data"]
    y = data["label"]
    with open(model_file, 'rb') as f:
        svclassifier_r = pickle.load(f)
    print("Loaded.")
    SVM_score = svclassifier_r.score(X, y)
    print("accuracy by score function: " + str(SVM_score))
    ##### CALCULATE METRIC:
    y_pred = svclassifier_r.predict(X)
    acc, pre, rec, f1 = calculate_metric(y, y_pred)
    # print("accuracy: " + str(acc))
    print("precision: " + str(pre))
    print("recall: " + str(rec))
    print("f1 score: " + str(f1))
    return SVM_score

def eval_visual(data, model_file):
    with open(data, 'rb') as f:
        info_dict = pickle.load(f)
    features_= info_dict['features']
    labels_ = info_dict['labels']

    with open(model_file, 'rb') as f:
        svclassifier_r = pickle.load(f)
    SVM_score = svclassifier_r.score(features_, labels_)
    print("accuracy by score function: " + str(SVM_score))
    ##### CALCULATE METRIC:
    y_pred = svclassifier_r.predict(features_)
    acc, pre, rec, f1 = calculate_metric(labels_, y_pred)
    # print("accuracy: " + str(acc))
    print("precision: " + str(pre))
    print("recall: " + str(rec))
    print("f1 score: " + str(f1))
    return SVM_score

def eval_headposes(data, model_file):
    with open(model_file, 'rb') as f:
        model = pickle.load(f)
    with open(data, 'rb') as f:
        data_ = pickle.load(f)

    videos_real, videos_fake, video_list, label_list, R_vec_feat, R_mat_feat, R_mat_full_feat, t_vec_feat = process_training_data(data_)
    features = [np.concatenate([R_mat_full_feat[i], t_vec_feat[i]]) for i in range(len(R_mat_feat))]
    X_test, y_test = shuffle(features, label_list, random_state=0)
    model_acc = model[0].score(X_test, y_test)
    ##### CALCULATE METRIC:
    y_pred = model[0].predict(X_test)
    acc, pre, rec, f1 = calculate_metric(y_test, y_pred)
    print("accuracy: " + str(acc))
    print("precision: " + str(pre))
    print("recall: " + str(rec))
    print("f1 score: " + str(f1))
    return acc

ckcpoint_spectrum = "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/ff/all/spectrum/c_2.000000"
saved_feature_spectrum = "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/spectrum/output_features/ff_all/test.pkl"
for ckcpoint_fold in sorted(glob(join(ckcpoint_spectrum, '*/model.pkl')), key=lambda fold: int(fold.replace('/model.pkl', '')[-1])):
    print("\n********************************************************")
    print("ckcpoint fold: ", ckcpoint_fold)
    eval_spectrum(data=saved_feature_spectrum, model_file=ckcpoint_fold)

# ckcpoint_visual = "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/UADFV/visual_artifact/c_2.000000"
# saved_feature_visual = "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/visual_artifact/output_features/UADFV/test.pkl"
# for ckcpoint_fold in sorted(glob(join(ckcpoint_visual, '*/model.pkl')), key=lambda fold: int(fold.replace('/model.pkl', '')[-1])):
#     print("\n********************************************************")
#     print("ckcpoint fold: ", ckcpoint_fold)
#     eval_visual(data=saved_feature_visual, model_file=ckcpoint_fold)

# ckcpoint_headpose = "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/df_in_the_wildv6/headpose/c_2.000000"
# saved_feature_headpose = "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/headpose_forensic/output_features/df_in_the_wildv6/test.pkl"
# for ckcpoint_fold in sorted(glob(join(ckcpoint_headpose, '*/model.pkl')), key=lambda fold: int(fold.replace('/model.pkl', '')[-1])):
#     print("\n********************************************************")
#     print("ckcpoint fold: ", ckcpoint_fold)
#     eval_headposes(data=saved_feature_headpose, model_file=ckcpoint_fold)


********************************************************
ckcpoint fold:  /mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/ff/all/spectrum/c_2.000000/0.6134173461823573_fold_0/model.pkl
Loaded.
accuracy by score function: 0.6134173461823573
precision: 0.6255699943919907
recall: 0.6134173461823573
f1 score: 0.6038320757548893

********************************************************
ckcpoint fold:  /mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/ff/all/spectrum/c_2.000000/0.614529280948851_fold_1/model.pkl
Loaded.
accuracy by score function: 0.614529280948851
precision: 0.6231024798220978
recall: 0.614529280948851
f1 score: 0.6076990487758911

********************************************************
ckcpoint fold:  /mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/ff/all/spectrum/c_2.000000/0.6127996046454164_fold_2/model.pkl
Loaded.
accuracy by score function: 0.6127996046454164
